## Question 1

In [6]:
using JuMP, Clp, ECOS, SCS
m = Model()
@variable(m, 0 <= x1 <= 3)
@variable(m, 0 <= x2 <= 3)
@variable(m, 0 <= x3 <= 3)

@constraint(m, 2x1 >= x2 + x3)
@objective(m, Max, 5x1 - x2 + 11x3)


INFO: Precompiling module SCS.


5 x1 - x2 + 11 x3

### Solve using Clp

In [14]:
setsolver(m, ClpSolver())
@time(solve(m))
getobjectivevalue(m)

  0.000658 seconds (98 allocations: 6.109 KB)


48.0

### Solve using ECOS

In [13]:
setsolver(m, ECOSSolver())
@time(solve(m))
getobjectivevalue(m)

  2.014500 seconds (1.78 M allocations: 66.719 MB, 1.86% gc time)

ECOS 2.0.5 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  -2.250e+01  -8.440e+01  +1e+02  2e-01  3e-01  1e+00  1e+01    ---    ---    1  1  - |  -  - 
 1  -4.615e+01  -5.603e+01  +2e+01  2e-02  6e-02  7e-01  3e+00  0.8410  6e-02   0  0  0 |  0  0
 2  -4.726e+01  -4.850e+01  +3e+00  3e-03  8e-03  2e-01  4e-01  0.9283  7e-02   0  0  0 |  0  0
 3  -4.799e+01  -4.803e+01  +8e-02  1e-04  2e-04  7e-03  1e-02  0.9798  9e-03   1  0  0 |  0  0
 4  -4.800e+01  -4.800e+01  +9e-04  1e-06  3e-06  8e-05  1e-04  0.9890  1e-04   1  0  0 |  0  0
 5  -4.800e+01  -4.800e+01  +9e-06  1e-08  3e-08  9e-07  1e-06  0.9890  1e-04   1  0  0 |  0  0
 6  -4.800e+01  -4.800e+01  +1e-07  1e-10  3e-10  1e-08  1e-08  0.9890  1e-04   1  0  0 |  0  0

OPTIMAL (within feastol=3.3e-10, reltol=2.2e-09, abstol=1.0e-07).
Runtime: 

47.999999986810174

### Solve using SCS

In [15]:
setsolver(m, SCSSolver())
@time(solve(m))
getobjectivevalue(m)

  2.099890 seconds (1.29 M allocations: 51.083 MB, 1.08% gc time)
----------------------------------------------------------------------------
	SCS v1.1.8 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2015
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 9
eps = 1.00e-04, alpha = 1.80, max_iters = 20000, normalize = 1, scale = 5.00
Variables n = 3, constraints m = 7
Cones:	linear vars: 7
Setup time: 4.75e-05s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf      -nan      -inf      -nan       inf  3.08e-05 
   100| 8.00e-05  1.91e-04  8.48e-06 -4.80e+01 -4.80e+01  2.69e-15  9.45e-05 
   140| 4.49e-06  2.70e-06  1.09e-07 -4.80e+01 -4.80e+01  0.00e+00  1.21e-04 
-----------------------------

48.00006780505256

### Analysis
The CLP solver is by far the most accurate and fastest of the three solvers. This is probably because it is designed to solve only linear programs, while the other two solvers are more generic and can solve both linear programs and second order cone programs. (Note the SCS solver gets a higher objective value than CLP, but CLP found an optimal solution. This comes from numerical error causing SCS to find an infeasible solution)

x3 is in the infeasible region

In [20]:
getvalue(x3)

3.0000130627112176

## Question 2

### a)
#### Transformed model:
$A = \begin{bmatrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1 \\
\end{bmatrix}$

Since $z_1$ is unconstrained, we redefine it as $z_1 = y - w$ where $y \geq 0 , w \geq 0$
Then to make all the rest of the variables non-negative, we simply shift there value.

$z'_2 = z_2 + 1$

$z'_3 = z_3 + 1$

$z'_4 = z_4 + 2$

Then we turn the rest of the constraints into equalities using non-negative surplus and slack variables.

$-y + w + 6z'_2 - z'_3 + z'_4 - s_1 = 4$

$z'_3 + z'_4 + s_2 = 5$

$z'_2 + s_3 = 6$

$z'_3 + s_4 = 6$

$z'_4 + s_5 = 4$

$$
\alpha =1 
$$

### b)

#### Solving in non standard form

In [26]:
ns_m = Model()
@variable(ns_m, z1)
@variable(ns_m, -1 <= z2 <= 5)
@variable(ns_m, -1 <= z3 <= 5)
@variable(ns_m, -2 <= z4 <= 2)
@constraint(ns_m, -z1 + 6z2  - z3 + z4 >= -3)
@constraint(ns_m, 7z2 + z4 == 5)
@constraint(ns_m, z3 + z4 <= 2)
@objective(ns_m, Max, 3z1 - z2)
solve(ns_m)

:Optimal

In [29]:
println("z1 = ", getvalue(z1))
println("z2 = ", getvalue(z2))
println("z3 = ", getvalue(z3))
println("z4 = ", getvalue(z4))
println("objective = ", getobjectivevalue(m))

z1 = 8.571428571428571
z2 = 0.42857142857142855
z3 = -1.0
z4 = 2.0
objective = 48.00006780505256


#### Solving standard form

In [ ]:
A = [
-1 1 6 -1 1 -1
    
]
b = [5;]

## Question 3